<a href="https://colab.research.google.com/github/gkondayya/Hackathon/blob/master/model_no_log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from google.colab import files
uploaded = files.upload()

df1 = pd.read_csv('Train.csv')

#country = pd.read_excel('Country-Code.xlsx')
#country.head()
df1.shape
df1.isnull().sum()


Saving Train.csv to Train (2).csv


id                                   0
name                                12
host_id                              0
host_name                           13
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       6982
reviews_per_month                 6982
calculated_host_listings_count       0
availability_365                     0
dtype: int64

# Modelling

# 1. DATA CLEANING : Removing unneessary columns (except 'host_id', neighbourhood_group',latitude', 'longitude', 'room_type', 'price')

In [ ]:
df1.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [ ]:
X_enc_rt = pd.get_dummies(df1.room_type, prefix='Roomtype')
X_enc_rt.head()
X_enc_nhg = pd.get_dummies(df1.neighbourhood_group, prefix='NBG')

df11 = pd.concat([df1, X_enc_rt.drop('Roomtype_Shared room',axis='columns')], axis=1)
df11 = pd.concat([df11, X_enc_nhg.drop('NBG_Staten Island',axis='columns')], axis=1)

df11.shape



(34226, 22)

In [ ]:
#df2=df1.drop(['id', 'name', 'host_name', 'neighbourhood_group',
#       'neighbourhood','minimum_nights', 'number_of_reviews', 'last_review',
#       'reviews_per_month', 'calculated_host_listings_count',
#       'availability_365'], axis='columns')
# did not work we can only drop 8 columns at a time

In [ ]:
df2=df11.drop(['id', 'name', 'host_name',
       'neighbourhood','minimum_nights', 'number_of_reviews'], axis='columns')
df2.shape

(34226, 16)

In [ ]:
df3=df2.drop(['last_review','reviews_per_month', 'calculated_host_listings_count','availability_365'], axis='columns')

In [ ]:
df3.shape
#df2.columns

(34226, 12)

In [ ]:
df3.isnull().sum() # no null values in df3

host_id                     0
neighbourhood_group         0
latitude                    0
longitude                   0
room_type                   0
price                       0
Roomtype_Entire home/apt    0
Roomtype_Private room       0
NBG_Bronx                   0
NBG_Brooklyn                0
NBG_Manhattan               0
NBG_Queens                  0
dtype: int64

# dropping the column "room_type"

In [ ]:
df4 = df3.drop(['room_type'],axis='columns')
df4.head()

,host_id,neighbourhood_group,latitude,longitude,price,Roomtype_Entire home/apt,Roomtype_Private room,NBG_Bronx,NBG_Brooklyn,NBG_Manhattan,NBG_Queens
0,4271676,Manhattan,40.72217,-73.99481,120,0,1,0,0,1,0
1,11460768,Manhattan,40.80020,-73.96045,1500,1,0,0,0,1,0
2,267193767,Queens,40.75558,-73.89316,200,1,0,0,0,0,1
3,43392243,Staten Island,40.60110,-74.07830,30,0,0,0,0,0,0
4,16065171,Manhattan,40.81022,-73.94266,115,0,0,0,0,1,0


# 2 REMOVING OUTLIERS

In [ ]:
df4.describe()

,host_id,latitude,longitude,price,Roomtype_Entire home/apt,Roomtype_Private room,NBG_Bronx,NBG_Brooklyn,NBG_Manhattan,NBG_Queens
count,3.422600e+04,34226.000000,34226.000000,34226.000000,34226.000000,34226.000000,34226.000000,34226.000000,34226.000000,34226.000000
mean,6.733847e+07,40.729059,-73.952070,153.480979,0.520920,0.455619,0.022410,0.410010,0.443902,0.116432
std,7.833091e+07,0.054604,0.046161,243.312608,0.499569,0.498034,0.148015,0.491842,0.496850,0.320747
min,2.438000e+03,40.499790,-74.244420,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.824440e+06,40.690040,-73.983150,69.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.065628e+07,40.722995,-73.955700,107.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.074344e+08,40.763200,-73.935982,175.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000
max,2.743115e+08,40.913060,-73.712990,10000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Outlier Removal (Retaining 5% -95% quantile prices) 

In [ ]:

max = df4['price'].quantile(0.95)
min = df4['price'].quantile(0.05)

df6 = df4[(df4['price']<max) &(df4['price']>min) ]
#df6 = remove_price_outliers(df5)
df6.head()
print('max,min', min,max)

max,min 40.0 360.0


In [ ]:
df7 = df6.drop(['neighbourhood_group'],axis='columns')
df7.shape


(30560, 10)

# 3 LINEAR REGRESSION (RMSE calculation)

In [ ]:

X=df7.drop('price', axis='columns')
X.head()
X.shape

(30560, 9)

In [ ]:
y = df7.price
#y = np.log10(y)
y.shape
y.head()

0    120
2    200
4    115
5    225
6     60
Name: price, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=1000)



y_train.head()

33185     50
10046    230
3286     249
26582     70
17347     60
Name: price, dtype: int64

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
sklr = LinearRegression(normalize=False)
lm = sklr.fit(X_train,y_train)


score = sklr.score(X_test,y_test)

preds_test = sklr.predict(X_test)
preds_train = sklr.predict(X_train)

#preds_test = 10**preds_test
#preds_train = 10**preds_train
#y_test = 10**y_test
#y_train = 10**y_train

rmse_test = np.sqrt(mean_squared_error(y_test, preds_test))
rmse_train = np.sqrt(mean_squared_error(y_train, preds_train))

print("RMSE_test: %f" % (rmse_test))
print("RMSE_train: %f" % (rmse_train))

print ('Score:', score)

df7 = df4.drop(['neighbourhood_group'],axis='columns')
df7.shape
X=df7.drop('price', axis='columns')
X.head()
X.shape
y = df7.price
y.shape
y.head()
X1_train,X1_test,y1_train,y1_test=train_test_split(X,y,test_size=0.2, random_state=1000)

#lm = sklr.fit(X_train,y_train)


score = sklr.score(X1_test,y1_test)

preds_test = sklr.predict(X1_test)


rmse_test = np.sqrt(mean_squared_error(y1_test, preds_test))


print("RMSE_test_outliers: %f" % (rmse_test))

print ('Score:', score)


RMSE_test: 55.333414
RMSE_train: 53.901843
Score: 0.4065516388334466
RMSE_test_outliers: 219.337765
Score: 0.07279207822987599


Results without WITH LOG

RMSE: 1.393696

Train Value : [ 39.00 - 100.00 -  93.00 -  80.00  - 98.00]

Pretiction  : [ 58.29 - 107.01 - 142.65 - 88.08 - 88.26]

Test Value : [ 55.00 -  45.00 -  60.00 - 150.00 - 195.00]

Pretiction [ 72.38 - 60.98 - 106.95- 86.79 - 144.76]

Results without LOG

RMSE: 36.597191

Train Value : [ 39.00 100.00  93.00  80.00  98.00]

Pretiction  : [ 61.31 121.75 147.80  96.31  98.87 ]

Test Value : [ 55.00 - 45.00 - 60.00 - 150.00 - 195.00]

Pretiction [ 76.60 -   64.40 - 127.06 - 97.01 - 151.19]



 # 4 XG Boost regression (RMSE Calculation)

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train,y_train)
score = xg_reg.score(X_test,y_test)
print ('Score:', score)

preds_test = xg_reg.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, preds_test))
print("RMSE_test: %f" % (rmse_test))

preds_train = xg_reg.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, preds_train))
print("RMSE_train: %f" % (rmse_train))

#accuracy =accuracy_score(y_test, preds)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))
#y_test=10**y_test


df7 = df4.drop(['neighbourhood_group'],axis='columns')
df7.shape
X=df7.drop('price', axis='columns')
X.head()
X.shape
y = df7.price
y.shape
y.head()
# X1_train,X1_test,y1_train,y1_test=train_test_split(X,y,test_size=0.2, random_state=1000)

#lm = sklr.fit(X_train,y_train)


score = xg_reg.score(X1_test,y1_test)

preds_test = xg_reg.predict(X1_test)


rmse_test = np.sqrt(mean_squared_error(y1_test, preds_test))


print("RMSE_test_outliers: %f" % (rmse_test))


[13:16:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Score: -0.10389404164932348
RMSE_test: 75.467453
RMSE_train: 74.317689
RMSE_test_outliers: 232.467246


# Lasso 

In [ ]:
from sklearn.linear_model import Lasso
skls = Lasso()
lm = skls.fit(X_train,y_train)

score = skls.score(X_test,y_test)

preds_test = skls.predict(X_test)
preds_train = skls.predict(X_train)

rmse_test = np.sqrt(mean_squared_error(y_test, preds_test))
rmse_train = np.sqrt(mean_squared_error(y_train, preds_train))

print("RMSE_test: %f" % (rmse_test))
print("RMSE_train: %f" % (rmse_train))

print ('Score:', score)

df7 = df4.drop(['neighbourhood_group'],axis='columns')
df7.shape
X=df7.drop('price', axis='columns')
X.head()
X.shape
y = df7.price
y.shape
y.head()
# X1_train,X1_test,y1_train,y1_test=train_test_split(X,y,test_size=0.2, random_state=1000)

#lm = sklr.fit(X_train,y_train)


score = skls.score(X1_test,y1_test)

preds_test = skls.predict(X1_test)


rmse_test = np.sqrt(mean_squared_error(y1_test, preds_test))


print("RMSE_test_outliers: %f" % (rmse_test))

RMSE_test: 56.367271
RMSE_train: 54.982657
Score: 0.384168323768168
RMSE_test_outliers: 220.314420


 # DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
skdt = DecisionTreeRegressor()
lm = skdt.fit(X_train,y_train)

score = skdt.score(X_test,y_test)

preds_test = skdt.predict(X_test)
preds_train = skdt.predict(X_train)

rmse_test = np.sqrt(mean_squared_error(y_test, preds_test))
rmse_train = np.sqrt(mean_squared_error(y_train, preds_train))

print("RMSE_test: %f" % (rmse_test))
print("RMSE_train: %f" % (rmse_train))

print ('Score:', score)


score = skdt.score(X1_test,y1_test)

preds_test = skdt.predict(X1_test)


rmse_test = np.sqrt(mean_squared_error(y1_test, preds_test))


print("RMSE_test_outliers: %f" % (rmse_test))

RMSE_test: 70.689533
RMSE_train: 0.000000
Score: 0.03145848986823241
RMSE_test_outliers: 214.073463


# Random forest with searchCV


In [ ]:
from sklearn.model_selection import  RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor ,RandomForestRegressor
rf_model = RandomForestRegressor(random_state=33)
param_grid = dict(n_estimators=range(15,100,15),max_depth=range(4,25),min_samples_split=range(60,500,20),min_samples_leaf=range(25,75,15),max_leaf_nodes=range(7,30,2))
grid_dt = RandomizedSearchCV(rf_model,param_grid,scoring='neg_mean_squared_error',n_jobs=-1,cv=5,random_state=33)
grid_dt.fit(X_train,y_train)
dt_pred1 = grid_dt.best_estimator_.predict(X_test)
print('RMSE_test:',np.sqrt(mean_squared_error(y_test, dt_pred1)))
dt_pred2 = grid_dt.best_estimator_.predict(X_train)
print('RMSE_train:',np.sqrt(mean_squared_error(y_train, dt_pred2)))
dt_pred3 = grid_dt.best_estimator_.predict(X1_test)
print('RMSE_test_outliers:',np.sqrt(mean_squared_error(y1_test, dt_pred3)))
X_test.shape

RMSE_test: 52.56529185957134
RMSE_train: 50.820376289003484
RMSE_test_outliers: 218.05706652762908


(6112, 9)

# Finding the best model using 'sklearn' GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import ShuffleSplit



In [ ]:
def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' :{
            'model' : LinearRegression(),
            'params': {
                'normalize' : [True, False]
            }
        },
        'lasso' : {
            'model' : Lasso(),
            'params': {
                'alpha' : [1,2],
                'selection' : ['random','cyclic']
            }
        },
        'decision_tree' : {
            'model' : DecisionTreeRegressor(),
            'params' : {
                'criterion' : ['mse', 'friedman_mse'],
                'splitter'  : ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits = 5, test_size = 0.2, random_state = 0 )
    for algo_name, config in algos.items():
        gs = GridSearchCV(config['model'],config['params'],cv=cv,return_train_score = False)
        gs.fit(X,y)
        scores.append({
            'model' : algo_name,
            'best_score' : gs.best_score_,
            'best_params' : gs.best_params_,
        })
    return pd.DataFrame(scores,columns = ['model', 'best_score', 'best_params'])

find_best_model_using_gridsearchcv(X,y)

,model,best_score,best_params
0,linear_regression,0.093290,{'normalize': True}
1,lasso,0.086391,"{'alpha': 1, 'selection': 'random'}"
2,decision_tree,-0.818443,"{'criterion': 'friedman_mse', 'splitter': 'best'}"


# Predictions on TEST data

In [ ]:
from google.colab import files
uploaded = files.upload()

df_t = pd.read_csv('Test.csv')


Saving Test.csv to Test (2).csv


In [ ]:
X_enc_rt = pd.get_dummies(df_t.room_type, prefix='Roomtype')
X_enc_rt.head()
X_enc_nhg = pd.get_dummies(df_t.neighbourhood_group, prefix='NBG')

df12 = pd.concat([df_t, X_enc_rt.drop('Roomtype_Shared room',axis='columns')], axis=1)
df12 = pd.concat([df12, X_enc_nhg.drop('NBG_Staten Island',axis='columns')], axis=1)

df12.shape



(14669, 21)

In [ ]:
df2=df12.drop(['id', 'name', 'host_name',
       'neighbourhood','minimum_nights', 'number_of_reviews'], axis='columns')
df3=df2.drop(['last_review','reviews_per_month', 'calculated_host_listings_count','availability_365'], axis='columns')
df4 = df3.drop(['room_type'],axis='columns')
df4.head()
X = df4.drop(['neighbourhood_group'],axis='columns')
X.shape
dt_pred = grid_dt.best_estimator_.predict(X)


final_submission = pd.DataFrame(dt_pred)

final_submission.head()